In [ ]:
import os
import sys
os.environ["PYSPARK_SUBMIT_ARGS"]='--conf spark.sql.catalogImplementation=in-memory pyspark-shell'
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

In [ ]:
spark

In [ ]:
from IPython.display import Image

## Pipeline

+ **DataFrame**: This ML API uses DataFrame from Spark SQL as an ML dataset, which can hold a variety of data types. E.g., a DataFrame could have different columns storing text, feature vectors, true labels, and predictions.


+ **Transformer**: A Transformer is an algorithm which can transform one DataFrame into another DataFrame. E.g., an ML model is a Transformer which transforms a DataFrame with features into a DataFrame with predictions.


+ **Estimator**: An Estimator is an algorithm which can be fit on a DataFrame to produce a Transformer. E.g., a learning algorithm is an Estimator which trains on a DataFrame and produces a model.


+ **Pipeline**: A Pipeline chains multiple Transformers and Estimators together to specify an ML workflow.


+ **Parameter**: All Transformers and Estimators now share a common API for specifying parameters.

In [ ]:
Image("pics/ml-Pipeline.png")

In [ ]:
Image("pics/ml-PipelineModel.png")

In [ ]:
from pyspark.ml.linalg import Vectors

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
training = spark.createDataFrame([
    (1.0, Vectors.dense([0.0, 1.1, 0.1])),
    (0.0, Vectors.dense([2.0, 1.0, -1.0])),
    (0.0, Vectors.dense([2.0, 1.3, 1.0])),
    (1.0, Vectors.dense([0.0, 1.2, -0.5]))], schema = ["label", "features"])

In [ ]:
lr = LogisticRegression(maxIter=10, regParam=0.01)

In [ ]:
lr

In [ ]:
lr.params

In [ ]:
lr.getOrDefault("regParam")

In [ ]:
model = lr.fit(training)

In [ ]:
type(model)

In [ ]:
model.coefficients

In [ ]:
model.interceptVector

In [ ]:
predict = model.transform(training)

In [ ]:
predict.show(1, truncate=False, vertical=True)

In [ ]:
model.getOrDefault("threshold")

## Toxic Comment Classification Challenge

In [ ]:
from pyspark.sql.types import *

In [ ]:
schema = StructType([
    StructField("id", StringType()),
    StructField("comment_text", StringType()),
    StructField("toxic", IntegerType()),
    StructField("severe_toxic", IntegerType()),
    StructField("obscene", IntegerType()),
    StructField("threat", IntegerType()),
    StructField("insult", IntegerType()),
    StructField("identity_hate", IntegerType())
])

In [ ]:
dataset = spark.read.csv("/user/pavel.klemenkov/lectures/lecture03/data/train.csv", schema=schema, header=True)

In [ ]:
dataset.show(2, vertical=True)

## There is an issue with multiline CSVs, fixed in 2.2.0 https://issues.apache.org/jira/browse/SPARK-19610

In [ ]:
!head -n3 toxic_comment/train.csv

In [ ]:
dataset = spark.read.csv("/user/pavel.klemenkov/lectures/lecture03/data/train.csv",
                         schema=schema, header=True, multiLine=True)

In [ ]:
dataset.select("id").show(10)

## You need to add `escape` parameter!

In [ ]:
dataset = spark.read.csv("/user/pavel.klemenkov/lectures/lecture03/data/train.csv",
                         schema=schema, header=True, multiLine=True, escape='"')

In [ ]:
dataset.select("id").show(10)

In [ ]:
dataset.show(2, vertical=True, truncate=False)

In [ ]:
dataset.rdd.getNumPartitions()

In [ ]:
dataset.count()

In [ ]:
dataset = dataset.repartition(4).cache()

In [ ]:
dataset.count()

## Let's define a binary target (toxic/non-toxic)

In [ ]:
from pyspark.sql import functions as f

In [ ]:
target = f.when(
    (dataset.toxic == 0) &
    (dataset.severe_toxic == 0) &
    (dataset.obscene == 0) &
    (dataset.threat == 0) &
    (dataset.insult == 0) &
    (dataset.identity_hate == 0),
    0
).otherwise(1)

In [ ]:
dataset = dataset.withColumn("target", target)

In [ ]:
dataset.select("id", "target").show(10)

In [ ]:
dataset.groupBy("target").count().collect()

In [ ]:
16225 / (16225 + 143346)

In [ ]:
dataset = dataset.drop("toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate").cache()

In [ ]:
dataset

In [ ]:
dataset.show(2, False, True)

In [ ]:
dataset.write.parquet("/user/pavel.klemenkov/lectures/lecture03/data/dataset", mode="overwrite")

## Let's fit the simplest binary-BoW logistic regression

In [ ]:
from pyspark.ml.feature import *

## Split comments into words

In [ ]:
tokenizer = Tokenizer(inputCol="comment_text", outputCol="words")

In [ ]:
dataset2 = tokenizer.transform(dataset)

In [ ]:
dataset2.select("id", "words").show(2, False, True)

In [ ]:
type(dataset2.take(1)[0].words)

## Convert texts into binary vectors using Hashing trick

In [ ]:
hasher = HashingTF(numFeatures=100, binary=True, inputCol=tokenizer.getOutputCol(), outputCol="word_vector")
dataset2 = hasher.transform(dataset2)

In [ ]:
dataset2.select("id", "word_vector").show(2, False, True)

## Now let's split into train and test. Don't forget that we have imbalanced classes, so let's do stratified sampling

In [ ]:
train = dataset2.sampleBy("target", fractions={0: 0.8, 1: 0.8}, seed=5757)

In [ ]:
train.groupby("target").count().collect()

In [ ]:
12906 / (12906 + 114769)

In [ ]:
test = dataset2.join(train, on="id", how="leftanti")

In [ ]:
test.groupby("target").count().collect()

In [ ]:
3319 / (3319 + 28577)

In [ ]:
train.rdd.getNumPartitions()

In [ ]:
test.rdd.getNumPartitions()

In [ ]:
train

In [ ]:
train = train.drop("comment_text", "words").cache()

In [ ]:
test = test.drop("comment_text", "words").coalesce(4).cache()

## Let's fit logistic regression

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
lr = LogisticRegression(featuresCol=hasher.getOutputCol(), labelCol="target", maxIter=15)

In [ ]:
lr_model = lr.fit(train)

In [ ]:
lr_model

In [ ]:
predictions = lr_model.transform(test)

In [ ]:
predictions

In [ ]:
predictions.select("id", "target", "prediction", "probability", "rawPrediction").show(5, False, True)

In [ ]:
predictions.select("target", f.col("prediction").cast("int")).filter("target == prediction").count()

In [ ]:
predictions.count()

In [ ]:
print("Accuracy is {}".format(28668 / 31896))

In [ ]:
predictions.select("target", f.col("prediction").cast("int"))\
           .filter((f.col("target") == 1) & (f.col("prediction") == f.col("target")))\
           .count()

In [ ]:
predictions_pd = predictions.select("target", f.col("prediction").cast("int")).toPandas()

In [ ]:
predictions_pd.head()

In [ ]:
lr.getOrDefault("threshold")

In [ ]:
from sklearn.metrics import classification_report, precision_score

In [ ]:
print(classification_report(predictions_pd.target, predictions_pd.prediction))

## What if we want more sophisticated metrics?

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="target", metricName='areaUnderROC')

In [ ]:
evaluator.evaluate(predictions)

In [ ]:
evaluator.setParams(metricName="precision")

In [ ]:
evaluator.evaluate(predictions)

## `spark.ml.evaluation.BinaryClassificationEvaluator` supports only ROC AUC and PR AUC. What if we want more?

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="target", metricName="accuracy")

In [ ]:
evaluator.evaluate(predictions)

In [ ]:
evaluator = evaluator.setMetricName("weightedPrecision")

In [ ]:
evaluator.evaluate(predictions)

In [ ]:
evaluator = evaluator.setMetricName("weightedRecall")

In [ ]:
evaluator.evaluate(predictions)

## Let's define a pipeline!

In [ ]:
dataset = spark.read.parquet("/user/pavel.klemenkov/lectures/lecture03/data/dataset")

In [ ]:
dataset

In [ ]:
dataset.rdd.getNumPartitions()

In [ ]:
from pyspark.ml import Pipeline

In [ ]:
pipeline = Pipeline(stages=[
    tokenizer,
    hasher,
    lr
])

In [ ]:
train = dataset.sampleBy("target", fractions={0: 0.8, 1: 0.8}).cache()

In [ ]:
test = dataset.join(train, on="id", how="leftanti").cache()

In [ ]:
pipeline_model = pipeline.fit(train)

In [ ]:
predictions = pipeline_model.transform(test)

In [ ]:
evaluator = BinaryClassificationEvaluator(rawPredictionCol="prediction", labelCol="target", metricName='areaUnderROC')

In [ ]:
evaluator.evaluate(predictions)

## Okay, may be some more sophisticated stuff?

In [ ]:
from pyspark.ml.classification import GBTClassifier

In [ ]:
gbt = GBTClassifier(featuresCol=hasher.getOutputCol(), labelCol="target", maxIter=10)

In [ ]:
pipeline = Pipeline(stages=[
    tokenizer,
    hasher,
    gbt
])

In [ ]:
pipeline_model = pipeline.fit(train)

In [ ]:
predictions = pipeline_model.transform(test)

In [ ]:
evaluator.evaluate(predictions)

## Lets add more degrees of freedom

In [ ]:
pipeline_model.stages

In [ ]:
pipeline_model = pipeline.fit(train, params={hasher.numFeatures: 1000})

In [ ]:
pipeline_model.stages[1].extractParamMap()

In [ ]:
predictions = pipeline_model.transform(test)

In [ ]:
evaluator.evaluate(predictions)

## Let's remove stopwords

In [ ]:
stop_words = StopWordsRemover.loadDefaultStopWords("english")

In [ ]:
stop_words

In [ ]:
swr = StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol="words_filtered", stopWords=stop_words)

In [ ]:
hasher = HashingTF(numFeatures=1000, binary=True, inputCol=swr.getOutputCol(), outputCol="word_vector")

In [ ]:
pipeline = Pipeline(stages=[
    tokenizer,
    swr,
    hasher,
    lr
])

In [ ]:
pipeline_model = pipeline.fit(train)

In [ ]:
pipeline_model.stages

In [ ]:
predictions = pipeline_model.transform(test)

In [ ]:
evaluator.evaluate(predictions)

## Need moar features!

In [ ]:
import pyspark.sql.functions as f

In [ ]:
dataset.printSchema()

In [ ]:
dataset = dataset.withColumn("comment_length", f.length(dataset.comment_text))

In [ ]:
train = dataset.sampleBy("target", fractions={0: 0.8, 1: 0.8}).cache()
test = dataset.join(train, on="id", how="leftanti").cache()

In [ ]:
train

In [ ]:
assembler = VectorAssembler(inputCols=[hasher.getOutputCol(), "comment_length"], outputCol="features")

In [ ]:
lr = LogisticRegression(labelCol="target", maxIter=15)

In [ ]:
pipeline = Pipeline(stages=[
    tokenizer,
    swr,
    hasher,
    assembler,
    lr
])

In [ ]:
pipeline_model = pipeline.fit(train)

In [ ]:
pipeline_model.stages

In [ ]:
predictions = pipeline_model.transform(test)

In [ ]:
evaluator.evaluate(predictions)

In [ ]:
pipeline_model.stages[-1].coefficients[-1]

## Ok, how do you do it right!? https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge/discussion/52557

## Very funny, anyway?

In [ ]:
count_vectorizer = CountVectorizer(inputCol=swr.getOutputCol(), outputCol="word_vector", binary=True)

In [ ]:
assembler = VectorAssembler(inputCols=[count_vectorizer.getOutputCol(), "comment_length"], outputCol="features")

In [ ]:
pipeline = Pipeline(stages=[
    tokenizer,
    swr,
    count_vectorizer,
    assembler,
    lr
])

In [ ]:
pipeline_model = pipeline.fit(train)

In [ ]:
predictions = pipeline_model.transform(test)

In [ ]:
evaluator.evaluate(predictions)

In [ ]:
Image("pics/Obama_not_bad.png")

## Hyperparameter tuning

In [ ]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [ ]:
paramGrid = ParamGridBuilder().addGrid(count_vectorizer.vocabSize, [100, 500])\
                              .addGrid(lr.regParam, [0.01, 0.05])\
                              .build()

In [ ]:
paramGrid

In [ ]:
crossval = CrossValidator(estimator=pipeline, estimatorParamMaps=paramGrid,
                          evaluator=evaluator, numFolds=3, parallelism=4)

In [ ]:
cv_model = crossval.fit(train)

In [ ]:
cv_model.avgMetrics

In [ ]:
cv_model.bestModel

In [ ]:
predictions = cv_model.transform(test)

In [ ]:
evaluator.evaluate(predictions)

In [ ]:
spark.stop()